In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

Способы ввода данных:
1. Ввод с клавиатуры
2. Случайная генерация
3. Импорт файла csv

In [203]:
from prettytable import PrettyTable
from scipy.optimize import linprog
import matplotlib.pyplot as plt
from fractions import Fraction
import random, scipy, csv, os
import pandas as pd
import numpy as np

In [205]:
try:
    way = int(input('Способ ввода данных: '))

    # Ввод с клавиатуры
    if way == 1:
        sum_strategy_A = int(input('Введите количество стратегий для компании А: '))
        name_rows, name_columns = [], []
        for num in range(sum_strategy_A):
            name = input('Введите название {} стратегии компании А: '.format(num + 1))
            name_rows.append(name)

        sum_strategy_B = int(input('Введите количество стратегий для компании B: '))
        for num in range(sum_strategy_B):
            name = input('Введите название {} стратегии компании B: '.format(num + 1))
            name_columns.append(name)

        matrix = []
        rows = []
        for num_row, row in enumerate(name_rows, start=1):
            for num_column, column in enumerate(name_columns, start=1):
                value = int(input('Введите элемент строки {} столбца {}: '.format(num_row, num_column)))
                rows.append(value)
            matrix.append(rows)
            rows = []

    # Случайная генерация
    elif way == 2:
        size_rows = int(input("Количество стратегий игрока А: ")) 
        size_cols = size_rows

        # Последовательно генерируем случайные числа типа int в диапазоне (0;1000) и добавляем их в матрицу
        matrix = []
        for i in range(size_rows):
            rows = []
            for j in range(size_cols):
                x = random.randint(0,1000)
                rows.append(x)
            matrix.append(rows)

        # список для имен строк и столбцов матрицы:
        name_rows, name_columns = [], []
        for i in range(size_rows):
            name_rows.append(i+1)

        for i in range(size_cols):
            name_columns.append(i+1)

    # файл csv
    elif way == 3:
        matrix = []
        file_way = input('Введите путь к файлу: ')
        with open(file_way, 'r', newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=';')
            for row in spamreader:
                matrix.append(row)

        # Для образования списка с названиями колонок матрицы
        name_columns = matrix[0]
        name_columns = name_columns[1:]
        del matrix[0]

        # Для образования списка с названиями строк
        name_rows = []
        for num, value in enumerate(matrix):
            name_rows.append(value[0])
            del matrix[num][0]

        # Перевод коэффициентов из str в тип int
        for num_1, row in enumerate(matrix):
            for num_2, value in enumerate(row):
                matrix[num_1][num_2] = int(value)


    matrix = pd.DataFrame(matrix)
    matrix.columns = name_columns
    matrix.index = name_rows
    print('\nВесовая матрица:\n', matrix, '\n')



    # ---------------------------------------------------
    # Оптимальная чистая стратегия

    def pure_strategy(matrix, name_columns, name_rows):
        max_min = list(matrix.min(axis=1))
        min_max = list(matrix.max())

        index_row = max_min.index(max(max_min))
        index_column = min_max.index(min(min_max))

        print('Оптимальная чистая стратегия для игрока А: ', name_rows[index_row])
        print('Цена игры для игрока А при выборе чистой оптимальной стратегии: ', max(max_min))

        print('Оптимальная чистая стратегия для игрока B: ', name_columns[index_column])
        print('Цена игры для игрока B при выборе чистой оптимальной стратегии: ', min(min_max))

    pure_strategy = pure_strategy(matrix, name_columns, name_rows)



    # ---------------------------------------------------
    # Оптимальная смешанная стратегия

    def nash_equilibrium(a, name_columns, name_rows):
        c = np.array([1 for x in range(len(a[0]))])
        b = np.array([1 for x in range(len(a))])

        bounds = [(0, None)] 
        for x in range(len(a[0]) - 1): 
            bounds += [(0, None)] 

        a_T = -np.transpose(a) 
        res = linprog(c, A_ub=a_T, b_ub= -b, bounds=bounds, options = {"disp": False}) 
        p = [res.x[i]/(res.fun) for i in range(len(res.x))] 

        print('\nЦена игры для игрока А при выборе смешанной оптимальной стратегии: ', round(1/res.fun, 4))

        # Таблица смешанной стратегии для игрока А
        mix_strategy_A = []
        for i in range(len(p)): 
            mix_strategy_A.append(str(round(p[i]*100)) + '%')

        mytable = PrettyTable()
        mytable.field_names = name_rows # имена полей таблицы
        mytable.add_row(mix_strategy_A)     # добавление данных по одной строке за раз
        print('Таблица смешанных стратегий для игрока А:\n', mytable)

        # Таблица смешанной стратегии для игрока В
        res = linprog(-c, A_ub=a, b_ub=b, bounds=bounds, options = {"disp": False}) 
        q = [res.x[i]/(-res.fun) for i in range(len(res.x))] 

        # Вывод
        mix_strategy_B = []
        for i in range(len(q)): 
            mix_strategy_B.append(str(round(q[i]*100)) + '%')

        mytable = PrettyTable()
        mytable.field_names = name_columns # имена полей таблицы
        mytable.add_row(mix_strategy_B)     # добавление данных по одной строке за раз
        print('Таблица смешанных стратегий для игрока B:\n', mytable)


    a = np.array(matrix)
    nash_equilibrium = nash_equilibrium(a, name_columns, name_rows)

except:
    print('\nЧто-то пошло не так.. Перезапустите программу')

Способ ввода данных: 2
Количество стратегий игрока А: 3

Весовая матрица:
      1    2    3
1  501  995  884
2  195  110  832
3  529  548  238 

Оптимальная чистая стратегия для игрока А:  1
Цена игры для игрока А при выборе чистой оптимальной стратегии:  501
Оптимальная чистая стратегия для игрока B:  1
Цена игры для игрока B при выборе чистой оптимальной стратегии:  529

Цена игры для игрока А при выборе смешанной оптимальной стратегии:  516.9098
Таблица смешанных стратегий для игрока А:
 +-----+----+-----+
|  1  | 2  |  3  |
+-----+----+-----+
| 43% | 0% | 57% |
+-----+----+-----+
Таблица смешанных стратегий для игрока B:
 +-----+----+----+
|  1  | 2  | 3  |
+-----+----+----+
| 96% | 0% | 4% |
+-----+----+----+
